In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
data=pd.read_csv("amazon_reviews.csv")

In [ ]:
data.head()

In [ ]:
#df=pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False,nrows=100000)

In [ ]:
####Selecting only required columns
data=data[['review_body','star_rating']]

In [ ]:
data=data.dropna()
data = data.reset_index(drop=True)
data

In [ ]:
#The reviews with star rating 4,5 are labelled as positive reviews and 1,2 are labelled as negative reviews. 
#Remove the reviews with star rating 3 as they are considered as neutral.

In [ ]:
data['star_rating']=data['star_rating'].astype(int) #convert the star_rating column to int
data=data[data['star_rating']!=3]
data['label']=np.where(data['star_rating']>=4,1,0) #1-Positve,0-Negative

In [ ]:
##### Number of reviews by start rating

In [ ]:
data['star_rating'].value_counts()

In [ ]:
##### Preprocessing

In [ ]:
### Lower casing
data['pre_process'] = data['review_body'].apply(lambda x: ' '.join(x.lower() for x in str(x).split()))

In [ ]:
data.head()

In [ ]:
####Remove the HTML tags and URLs from the reviews.

In [ ]:
from bs4 import BeautifulSoup
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
import re
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r'http\S+', '', x))

In [ ]:
#Perform the Contractions on the reviews.
#Example: it won’t be converted as it will not be

data.head()

In [ ]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"would't", "would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

In [ ]:
data.head()

In [ ]:
#Remove non-alpha characters

In [ ]:
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

In [ ]:
data.head(20)

In [ ]:
##Remove the extra spaces between the words

In [ ]:
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(' +', ' ', x))

In [ ]:
#### Remove stopwords

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

In [ ]:
data.head()

In [ ]:
#### Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [ ]:
data.head()

In [ ]:
###Split the Data into Training and Testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['pre_process'], data['label'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['review_body'], data['label'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

In [ ]:
####Feature extraction
##### TF-IDF vectorizer

In [ ]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [ ]:
tf_x_train

In [ ]:
tf_x_train

In [ ]:
##### using SVM algorithm

from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [ ]:
###Fitting the Training data into model

In [ ]:
clf.fit(tf_x_train,Y_train)

In [ ]:
###Predicting the Test data

In [ ]:
y_test_pred=clf.predict(tf_x_test)

In [ ]:
y_test_pred

In [ ]:
#Analyzing the results

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [ ]:
report

In [ ]:
###### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver='saga')

In [ ]:
clf.fit(tf_x_train,Y_train)

In [ ]:
y_test_pred=clf.predict(tf_x_test)

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [ ]:
report